데이터 전처리

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("../"))

from utils import *
from MLP import train_and_evaluate
from permutation import permutation

# Load dataset
region="andong"
path = f"../../collect_data/filtered/kma/merged/kma_{region}_meta.csv"

# 2. CSV 파일 읽어서 데이터프레임으로 변환
korea_data = pd.read_csv(path)
china_data = concat_china_data(korea_data)

korea_data = convert_timesteps(korea_data)
print_missing_info(korea_data)

china_data = convert_timesteps(china_data)
print_missing_info(china_data)

(40177, 22)
Series([], dtype: float64)
Index(['TM', 'STN', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'PV', 'WW',
       'CA_TOT', 'CA_MID', 'CT', 'VS', 'TS', 'IR', 'PM10', '지점', '위도', '경도',
       'diff'],
      dtype='object')
(9485, 107)
VV             0.031629
WD_yanan       0.021086
WS_yanan       0.021086
TA_yanan       0.105430
TD_yanan       0.274117
HM_yanan       0.685293
PS_yanan       0.031629
PR             0.063258
VV_tongliao    0.010543
WD_tongliao    0.010543
WS_tongliao    0.010543
TA_tongliao    0.137059
TD_tongliao    0.231945
HM_tongliao    0.242488
PS_tongliao    0.010543
PR_tongliao    0.010543
VV_qingdao     0.094887
WD_qingdao     0.115973
WS_qingdao     0.115973
TA_qingdao     0.094887
TD_qingdao     0.200316
HM_qingdao     0.790722
PS_qingdao     0.094887
PR_qingdao     0.094887
VV_chifeng     0.063258
WD_chifeng     0.021086
WS_chifeng     0.063258
TA_chifeng     0.115973
TD_chifeng     0.263574
HM_chifeng     0.179230
PS_chifeng     0.031629
PR_chifeng     0

MLP-Model - 한국만

In [2]:
removed_columns=['TM', 'PM10','WW','CT','diff']

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)



Cross-validated Mean Absolute Error: 12.611796188354493
252/252 [==============================] - 0s 526us/step - loss: 228.0146 - mae: 10.9808
Test Set Mean Absolute Error: 10.98078441619873


질적 분석

In [3]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_korea, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 9088.8330078125
252/252 [==============================] - 0s 416us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: 0.17352324414041503
Feature: WS, Importance: -0.08526217880262266
Feature: PA, Importance: 20.633271946050446
Feature: PS, Importance: 209.1230650092084
Feature: TA, Importance: 6.1239548636367545
Feature: TD, Importance: 0.047634100767572816
Feature: HM, Importance: 25.31987078784332
Feature: PV, Importance: -0.3865091549367207
Feature: CA_TOT, Importance: 0.16304008621627872
Feature: CA_MID, Importance: 1.1478991031724946
Feature: VS, Importance: 248.75131084497335
Feature: TS, Importance: -3.6501530368179376
Feature: IR, Importance: 0.0736364371898162
Feature: 지점, Importance: 0.0
Feature: 위도, Importance: 0.0
Feature: 경도, Importance: 0.0

Top 10 Important Features (Sorted Descending by Importance Score):
Feature: VS, Importance: 248.75131084497335
Feature: PS, Importance: 209.1230650092084
Feature: HM, Importance: 25.3

MLP model - 중국 데이터 포함

In [4]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 13.246943855285645
60/60 [==============================] - 0s 559us/step - loss: 161.5029 - mae: 9.2125
Test Set Mean Absolute Error: 9.212471961975098


질적 분석

In [5]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_china, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 17145.3046875
60/60 [==============================] - 0s 441us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: 1.6611728278803638e-05
Feature: WS, Importance: 8.751481436775066e-06
Feature: PA, Importance: -6.026298797223717e-06
Feature: PS, Importance: 5.0832598572014834e-05
Feature: TA, Importance: 3.359668335178867e-05
Feature: TD, Importance: -4.4722837628796696e-07
Feature: HM, Importance: 2.0638374917325563e-05
Feature: PV, Importance: 3.140559419989586e-05
Feature: CA_TOT, Importance: 1.2779737517121248e-05
Feature: CA_MID, Importance: 1.5907105262158438e-05
Feature: VS, Importance: 0.009289868905761977
Feature: TS, Importance: 1.8008104962063954e-05
Feature: IR, Importance: -3.504454143694602e-05
Feature: 지점, Importance: 0.0
Feature: 위도, Importance: 0.0
Feature: 경도, Importance: 0.0
Feature: STN_yanan, Importance: 0.0
Feature: IW, Importance: 0.0
Feature: IR_yanan, Importance: 0.0
Feature: IX, Importance: 0.0
Feature: VV, Impo

## MLP 한국만
Feature Importance 음수인 feature 제거한 버전

In [6]:
print(low_importance_features_korea)

['STN', 'WS', 'PV', 'TS', '지점', '위도', '경도']


In [7]:
removed_columns=['TM', 'PM10','WW','CT','diff']
removed_columns+=low_importance_features_korea

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 12.51886444091797
252/252 [==============================] - 0s 597us/step - loss: 278.6973 - mae: 12.2039
Test Set Mean Absolute Error: 12.203887939453125


In [8]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 11085.54296875
252/252 [==============================] - 0s 445us/step
Feature: WD, Importance: 0.11659635790256288
Feature: PA, Importance: 23.45517697535961
Feature: PS, Importance: 224.42216461460575
Feature: TA, Importance: 1.2353911811742364
Feature: TD, Importance: 0.43240024561982865
Feature: HM, Importance: 22.43523983596606
Feature: CA_TOT, Importance: 0.13328633749952132
Feature: CA_MID, Importance: 1.1021706434145018
Feature: VS, Importance: 265.7529159177746
Feature: IR, Importance: -0.055671520455507564

Top 10 Important Features (Sorted Descending by Importance Score):
Feature: VS, Importance: 265.7529159177746
Feature: PS, Importance: 224.42216461460575
Feature: PA, Importance: 23.45517697535961
Feature: HM, Importance: 22.43523983596606
Feature: TA, Importance: 1.2353911811742364
Feature: CA_MID, Importance: 1.1021706434145018
Feature: TD, Importance: 0.43240024561982865
Feature: CA_TOT, Importance: 0.13328633749952132
Feature: W

## MLP 중국포함
Feature Importance 음수인 feature 제거한 버전

In [9]:
print(low_importance_features_china)

['STN', 'PA', 'TD', 'IR', '지점', '위도', '경도', 'STN_yanan', 'IW', 'IR_yanan', 'IX', 'VV', 'RH', 'PM10_yanan', 'LON', 'LAT', 'IW_tongliao', 'IR_tongliao', 'IX_tongliao', 'VV_tongliao', 'WS_tongliao', 'PT_tongliao', 'PR_tongliao', 'RH_tongliao', 'LON_tongliao', 'LAT_tongliao', 'IW_qingdao', 'IR_qingdao', 'IX_qingdao', 'WS_qingdao', 'HM_qingdao', 'PR_qingdao', 'RH_qingdao', 'PM10_qingdao', 'LON_qingdao', 'LAT_qingdao', 'IW_chifeng', 'IR_chifeng', 'IX_chifeng', 'VV_chifeng', 'HM_chifeng', 'PS_chifeng', 'RH_chifeng', 'PM10_chifeng', 'LON_chifeng', 'LAT_chifeng', 'IW_dalian', 'IR_dalian', 'IX_dalian', 'VV_dalian', 'WS_dalian', 'HM_dalian', 'PT_dalian', 'RH_dalian', 'LON_dalian', 'LAT_dalian']


In [10]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']
removed_columns+=low_importance_features_china

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 13.218231391906738
60/60 [==============================] - 0s 664us/step - loss: 176.9925 - mae: 9.7915
Test Set Mean Absolute Error: 9.79153823852539


In [11]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 17200.732421875
60/60 [==============================] - 0s 449us/step
Feature: WD, Importance: 1.120479246310424e-05
Feature: WS, Importance: 1.5773581253597513e-05
Feature: PS, Importance: 14.40131030607954
Feature: TA, Importance: 1.2948653966304846e-05
Feature: HM, Importance: 0.20624649754572602
Feature: PV, Importance: 1.150964671978727e-05
Feature: CA_TOT, Importance: 6.970619142521173e-06
Feature: CA_MID, Importance: 5.010383756598458e-06
Feature: VS, Importance: 3.928954933636851
Feature: TS, Importance: 1.0909590855590068e-05
Feature: WD_yanan, Importance: 4.0636479025124575e-05
Feature: WS_yanan, Importance: 1.6436377336503938e-05
Feature: TA_yanan, Importance: 1.176141231553629e-05
Feature: TD_yanan, Importance: 1.5101132885320111e-05
Feature: HM_yanan, Importance: 7.615719732712023e-06
Feature: PS_yanan, Importance: 1.3580884478869848e-05
Feature: PT, Importance: -1.1285243090242148e-06
Feature: PR, Importance: 9.489888907410204e-06
